# Demande immobilière en France

Par Yanis Larré, Tom Croquette et Renan.


## Introduction 

Nous nous proposons ici d'étudier la demande immobilière en France. Mettre de la biblio + idées + NB


## Sommaire

* [Installation](#installation)


## I. Installation
La première étape du projet consiste à télécharger tous les modules et fonctions dont nous auront besoin.

In [30]:
!pip install -r requirements.txt

# Modules :
import os
import s3fs
import pandas as pd

# Fonctions :
from scripts import get_data
from scripts import data_clean
from scripts import data_analysis
from scripts import data_visualization
from scripts import do_ols

## II. Préparation des données

Nous téléchargeons dans cette parties les données utiles dans notre projet. Nous procédons également au nettoyage de celles-ci afin de pouvoir les exploiter convenablement.

### 1. Données immobilières

Nous décidons de récupérer des données foncières sur le site *data.gouv*, en utilisant plus particulièrement les données DVF (Demande de Valeur Foncière).
Ces données concernent la quasi-totalité de la France métropolitaine et des département d'Outre-Mer. En effet, les données de Mayotte ne sont pas incluses, ainsi que celles de 3 départements métropolitains : le Bas-Rhin (67), le Haut-Rhin (68) et la Moselle (57), car ces département utilisent un système hérité de l'annexion allemande appelé le Livre Foncier. Leurs données ne sont pas centralisées par la DGFiP de la même manière, donc elles ne redescendent pas dans la base nationale DVF.


In [17]:
df = get_data.get_dvf()

/home/onyxia/work/Python_2A/scripts/get_data.py:23: DtypeWarning: Columns (12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, dtype={'code_commune': "str"})


#### Nettoyage

Dans notre analyse, nous avons pris le parti de ne considérer que les maisons et appartements. En effet, la base de données contient d'autres catégories de biens immobiliers telles que les terrains vierges par exemple. Toutefois, les maisons et appartements sont les biens susceptibles de montrer les résultats les plus parlants ; ce sont pas ailleurs les catégories proposant les données le plus complètes sur le prix, la localisation et la surface notamment.

In [ ]:
df = df[df['type_local'].isin(['Maison', 'Appartement'])]
df = df.reset_index()

Ensuite, les codes communes n'ont pas tous le même type : certains sont de type *int*, d'autres de type *string*. On convertit l'ensemble au type *string* car certains codes communes (notamment ceux des communes corses, commençant par 2A ou 2B) ne peuvent pas être convertis en type *int*.

In [38]:
data_clean.convertir_codes_communes(df)
print(df['code_commune'])

0          01130
1          01451
2          01364
3          01053
4          01177
           ...  
6247210    75111
6247211    75112
6247212    75116
6247213    75113
6247214    75105
Name: code_commune, Length: 6247215, dtype: object


#### Aperçus des données

En calculant le nombre de ventes par commune, on remarque que Toulouse est la commune comptant le plus de ventes entre 2020 et 2025 (50684 ventes).

In [39]:
ventes_par_commune = df['code_commune'].value_counts().reset_index(name='nombre')

# Premiers résultats :
print(ventes_par_commune)
print( ventes_par_commune["nombre"][ventes_par_commune["code_commune"]=='2A004'])
print(ventes_par_commune['nombre'].max())
print(ventes_par_commune['nombre'].idxmax())
print(ventes_par_commune['code_commune'][ventes_par_commune['nombre'].idxmax()])
print(ventes_par_commune['nombre'][ventes_par_commune['nombre'].idxmax()])

      code_commune  nombre
0            31555   50684
1            06088   46697
2            44109   30331
3            33063   30077
4            34172   28927
...            ...     ...
33265        52549       1
33266        21136       1
33267        21653       1
33268        52053       1
33269        52505       1

[33270 rows x 2 columns]
145    4880
Name: nombre, dtype: int64
50684
0
31555
50684


### 2. Données de population

Afin d'étudier la dynamique autour du secteur immobilier (restreint aux maisons et aux appartements), il nous est nécessaire d'utiliser les données de population du territoire français. Pour cela, nous utilisons les données de XXX.

In [1]:
df_pop = get_data.get_pop()

NameError: name 'get_data' is not defined

Certaines valeurs ne sont toutefois pas renseignées ; on remarque aussi que certaines communes ont des populations très faibles.
Pour que le ratio des ventes sur la population ait du sens, il nous faut éliminer les communes ayant une population très faible (en particulier, la commune de Bezonvaux ne comptant aucun habitant...).
On imposte donc un nombre minimum d'habitants arbitraire, et on exclut les communes de population trop faible ou non renseignée.

In [ ]:
lignes_na = data_clean.filtre_donnes_pop(df_pop)
print(lignes_na)